# Rolling Planning
> Plannunghorizont je 3 Tage
> 
> Für 4 Planungstage

In [2]:
import json
import pandas as pd

# Basics
import utils.basics.presenter as show
import utils.basics.converter as convert

import utils.checker as check

from ProductionDaySimulation import ProductionDaySimulation

In [3]:
import utils.schedule_solver__tardiness_plus as ssv_t
import utils.reschedule.schedule_solver__tardiness_plus as rssv_t

In [4]:
import utils.rolling_planning.init_jobs_times as rp_init
import utils.rolling_planning.procedure as rp_proced

In [5]:
# Datei laden
with open("data/jobshop_instances.json", "r", encoding="utf-8") as f:
    jobshop_instances = json.load(f)

instance =  jobshop_instances["instance ft10"]
df_instance = convert.jssp_dict_to_df(instance)

df_jssp, df_times = rp_init.init_jobs_with_arrivals(df_instance, 6,  u_b_mmax = 0.9,
                                                   generate_deadlines = True,  deadlines_buffer_factor = 1.8)
df_jssp

,Job,Operation,Machine,Processing Time
0,Job_000,0,M0,29
1,Job_000,1,M1,78
2,Job_000,2,M2,9
3,Job_000,3,M3,36
4,Job_000,4,M4,49
...,...,...,...,...
1135,Job_113,5,M9,76
1136,Job_113,6,M5,47
1137,Job_113,7,M3,52
1138,Job_113,8,M4,90


In [6]:
df_times

,Job,Arrival,Deadline
0,Job_000,0.00,1604.019622
1,Job_001,88.05,2159.062676
2,Job_002,152.78,2459.319608
3,Job_003,207.97,2867.800006
4,Job_004,246.61,1842.508003
...,...,...,...
109,Job_109,8390.89,11050.720006
110,Job_110,8397.28,9993.178003
111,Job_111,8398.72,10705.259608
112,Job_112,8524.04,10128.059622


In [7]:
solver_limit = 60*30
solver_limit

1800

## I) Day 0

In [9]:
day_numb = 0

In [10]:
day_start = 1440*day_numb
day_end = day_start + 1440
planning_end = day_start + 3*1440

In [11]:
df_execution = None
df_undone = None

In [12]:
file_template = "data/{prefix}_schedule_{day:02d}.csv"
file_name = file_template.format(prefix="04b", day=day_numb)
file_name

'data/04b_schedule_00.csv'

In [ ]:
# I)
df_jssp_filtered, df_times_filtered = rp_proced.filter_jobs_by_arrival_window(df_times, df_jssp, day_start, planning_end)
df_jssp_filtered

# II)
if df_execution is not None:
    print("Exectution Dataframe is not None")
    df_jssp_filtered = rp_proced.get_unexecuted_operations(df_jssp_filtered, df_execution)

if (df_undone is not None) and (not df_undone.empty):
    print("Es gibt Operations in undone Dataframe")
    df_jssp_filtered = rp_proced.get_unexecuted_operations(df_jssp_filtered, df_undone)

if (df_execution is not None) | (df_undone is not None):
    print("Zeiten werden aktualisiert")
    df_times_filtered = rp_proced.update_times_after_operation_changes(df_times, df_jssp_filtered)

# III) 
if df_execution is not None:
    df_execution_important = rp_proced.get_operations_running_into_day(df_execution, day_start)
    

# Scheduling/Rescheduling

if df_execution is None: # Scheduling
    print("Scheduling startet ...")
    df_plan = ssv_t.solve_jssp_sum_tardiness(df_jssp_filtered, df_times_filtered, 
                                                  solver_time_limit = solver_limit)
else:                    # Rescheduling
    print("Rescheduling startet ...")
    df_plan = rssv_t.solve_jssp_sum_tardiness_with_fixed_ops(
        df_jssp_filtered, df_times_filtered, df_execution_important, 
        solver_time_limit = solver_limit,
        reschedule_start = day_start
    )

show.plot_gantt_machines(df_plan, title = f"Gantt-Diagramm ab Tag {day_numb}")
check.check_all_constraints(df_plan)
df_plan.to_csv(file_name, index = False)

Scheduling startet ...


In [ ]:
# Simulation (ein Tag)

filt = (df_plan["Start"] >=  day_start) & (df_plan["Start"] < day_end)
df_plan_single_day = df_plan[filt]
df_plan_single_day

In [ ]:
simulation = ProductionDaySimulation(df_plan_single_day, vc=0.35)
df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)
show.plot_gantt_machines(df_execution, title = f"Gantt-Diagramm für Simulationstag {day_numb}",
                         duration_column = "Simulated Processing Time")

## II) Day 1

In [ ]:
day_numb = 1

day_start = 1440*day_numb
day_end = day_start + 1440
planning_end = day_start + 3*1440

file_template = "data/{prefix}_schedule_{day:02d}.csv"
file_name = file_template.format(prefix="04b", day=day_numb)
file_name

In [ ]:
# I)
df_jssp_filtered, df_times_filtered = rp_proced.filter_jobs_by_arrival_window(df_times, df_jssp, day_start, planning_end)
df_jssp_filtered

# II)
if df_execution is not None:
    print("Exectution Dataframe is not None")
    df_jssp_filtered = rp_proced.get_unexecuted_operations(df_jssp_filtered, df_execution)

if (df_undone is not None) and (not df_undone.empty):
    print("Es gibt Operations in undone Dataframe")
    df_jssp_filtered = rp_proced.get_unexecuted_operations(df_jssp_filtered, df_undone)

if (df_execution is not None) | (df_undone is not None):
    print("Zeiten werden aktualisiert")
    df_times_filtered = rp_proced.update_times_after_operation_changes(df_times, df_jssp_filtered)

# III) 
if df_execution is not None:
    df_execution_important = rp_proced.get_operations_running_into_day(df_execution, day_start)
    

# Scheduling/Rescheduling

if df_execution is None: # Scheduling
    print("Scheduling startet ...")
    df_plan = ssv_t.solve_jssp_sum_tardiness(df_jssp_filtered, df_times_filtered, 
                                                  solver_time_limit = solver_limit)
else:                    # Rescheduling
    print("Rescheduling startet ...")
    df_plan = rssv_t.solve_jssp_sum_tardiness_with_fixed_ops(
        df_jssp_filtered, df_times_filtered, df_execution_important, 
        solver_time_limit = solver_limit,
        reschedule_start = day_start
    )

show.plot_gantt_machines(df_plan, title = f"Gantt-Diagramm ab Tag {day_numb}")
check.check_all_constraints(df_plan)
df_plan.to_csv(file_name, index = False)

# Simulation (ein Tag)

filt = (df_plan["Start"] >=  day_start) & (df_plan["Start"] < day_end)
df_plan_single_day = df_plan[filt]

simulation = ProductionDaySimulation(df_plan_single_day, vc=0.35)
df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)
show.plot_gantt_machines(df_execution, title = f"Gantt-Diagramm für Simulationstag {day_numb}",
                         duration_column = "Simulated Processing Time")

In [ ]:
-

In [ ]:
# I)
df_jssp_filtered, df_times_filtered = rp_proced.filter_jobs_by_arrival_window(df_times, df_jssp, day_start, planning_end)
df_jssp_filtered

# II)
if df_execution is not None:
    print("Exectution Dataframe is not None")
    df_jssp_filtered = rp_proced.get_unexecuted_operations(df_jssp_filtered, df_execution)

if (df_undone is not None) and (not df_undone.empty):
    print("Es gibt Operations in undone Dataframe")
    df_jssp_filtered = rp_proced.get_unexecuted_operations(df_jssp_filtered, df_undone)

if (df_execution is not None) | (df_undone is not None):
    print("Zeiten werden aktualisiert")
    df_times_filtered = rp_proced.update_times_after_operation_changes(df_times, df_jssp_filtered)

# III) 
if df_execution is not None:
    df_execution_important = rp_proced.get_operations_running_into_day(df_execution, day_start)
    

# Scheduling/Rescheduling

if df_execution is None: # Scheduling
    print("Scheduling startet ...")
    df_plan = ssv_a.solve_jssp_weighted_individual_flowtime(df_jssp_filtered, df_times_filtered, 
                                                  solver_time_limit = solver_limit)
else:                    # Rescheduling
    print("Rescheduling startet ...")
    df_plan = rssv_a.solve_jssp_weighted_individual_flowtime_with_fixed_ops(
        df_jssp_filtered, df_times_filtered, df_execution_important, 
        solver_time_limit = solver_limit,
        reschedule_start = day_start
    )

show.plot_gantt_machines(df_plan, title = f"Gantt-Diagramm ab Tag {day_numb}")
check.check_all_constraints(df_plan)
df_plan.to_csv(file_name, index = False)

# Simulation (ein Tag)

filt = (df_plan["Start"] >=  day_start) & (df_plan["Start"] < day_end)
df_plan_single_day = df_plan[filt]

simulation = ProductionDaySimulation(df_plan_single_day, vc=0.35)
df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)
show.plot_gantt_machines(df_execution, title = f"Gantt-Diagramm für Simulationstag {day_numb}",
                         duration_column = "Simulated Processing Time")